<a href="https://colab.research.google.com/github/Balajinanda/Datascience-Project/blob/main/Data_Science_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
import os

# Define the path to your zip file
zip_file_path = '/content/opportunity+activity+recognition.zip'
extract_dir = 'extracted_dataset/'

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Check the extracted files
extracted_files = os.listdir(extract_dir)
print(extracted_files)

['OpportunityUCIDataset']


In [2]:
# Example for reading a text-based .dat file
file_path = '/content/extracted_dataset/OpportunityUCIDataset/dataset/S1-ADL1.dat'

# Load the .dat file assuming it's a space-separated text file (adjust separator as needed)
with open(file_path, 'r') as file:
    data = file.readlines()

# Check the first few lines of the file
for line in data[:5]:
    print(line)

0 87 975 -287 11 1001 163 95 975 152 195 950 339 249 1008 99 62 924 -326 209 973 -41 -362 844 396 -237 1039 89 -345 885 329 -126 932 374 NaN NaN NaN -983 -199 119 -166 -50 70 483 -499 42 617 -475 457 430 -1016 -18 78 -189 -158 70 356 -242 -552 210 -635 136 731 -870 532 137 305 90 471 262 -26 -682 219 607 -84 -760 -982 114 5 -154 39 17 354 -338 582 673 -135 699 198 -920 -358 171 -145 47 48 389 202 593 741 -98 641 -172 119 -28 4 256 45 -142 -319 604 -953 -226 -876 2319 -876 226 2319 -76 83 -20 17 3 1 70 -219 319 -845 -20 57 42 57 20 42 175 281 -921 265 417 415 -72 682 -778 414 411 132 -862 508 420 427 46 -29 -995 415 418 -107 -771 -595 417 400 816 -114 -537 412 413 -44 24 -1004 413 417 -520 -90 880 419 419 200 -875 651 425 410 -57 47 -990 413 411 437 34 907 412 414 81 -972 236 416 414 0 0 0 0 0 0 0 0 0 0 0 0 0 1000 -63 19 -244 0 -960 -973 15 21 -1000 -68 -10 NaN NaN NaN 39 -86 -1010 40 39 -1080 -1008 32 -1023 5762 2685 1265 5858 2707 1134 6294 2663 1533 5789 2907 1447 0 0 0 0 0 0 0

33 1

In [4]:
import pandas as pd

# Load a .dat file into a pandas DataFrame
# Assuming space-separated values (adjust delimiter as needed)
df = pd.read_csv(file_path, delimiter=' ', header=None)

# Check the first few rows of the DataFrame
print(df.head())

import pandas as pd

# Step 1: Load the .dat file (assuming space-separated values)
file_path = '/content/extracted_dataset/OpportunityUCIDataset/dataset/S1-ADL1.dat'
df = pd.read_csv(file_path, delimiter=' ', header=None)

# Step 2: Data exploration
print(df.head())       # Display the first few rows
print(df.info())       # Check for missing values and data types
print(df.describe())   # Summary statistics

# Step 3: Handle missing values (if applicable)
df.fillna(df.mean(), inplace=True)

# Step 4: Normalize the data (if sensor data)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Assuming sensor data in columns (adjust column names as necessary)
sensor_columns = df.columns
df[sensor_columns] = scaler.fit_transform(df[sensor_columns])

# Step 5: Save the preprocessed data
df.to_csv('preprocessed_file.csv', index=False)

   0      1      2      3     4       5      6      7       8      9    ...  \
0    0   87.0  975.0 -287.0  11.0  1001.0  163.0   95.0   975.0  152.0  ...   
1   33  124.0  978.0 -389.0  -7.0  1014.0  199.0  124.0   968.0  123.0  ...   
2   67  102.0  996.0 -440.0 -49.0  1024.0  193.0  127.0  1001.0  113.0  ...   
3  100   59.0  861.0 -384.0  -9.0  1023.0  202.0  110.0  1007.0  106.0  ...   
4  133  119.0  946.0 -426.0 -22.0  1026.0  188.0   98.0  1001.0   92.0  ...   

      240     241     242  243  244  245  246  247  248  249  
0  5789.0  2907.0  1447.0    0    0    0    0    0    0    0  
1  5789.0  2908.0  1443.0    0    0    0    0    0    0    0  
2  5789.0  2910.0  1440.0    0    0    0    0    0    0    0  
3  5789.0  2912.0  1440.0    0    0    0    0    0    0    0  
4  5791.0  2915.0  1442.0    0    0    0    0    0    0    0  

[5 rows x 250 columns]
   0      1      2      3     4       5      6      7       8      9    ...  \
0    0   87.0  975.0 -287.0  11.0  1001.0  1